In [1]:
import matplotlib.pyplot as plt
from sargan_models import SARGAN
import os
import tensorflow as tf
from tqdm import tqdm
from random import shuffle
# import skimage.measure as ski_me
import time
import numpy as np
from cifar_helper import get_data, chunks
from sar_utilities import add_gaussian_noise
from alert_utilities import send_images_via_email

from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torch

/home/grad3/hle/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/grad3/hle/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/grad3/hle/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/grad3/hle/.local/lib/python3.5/site-packages/tensorflow/python/framework/dty

In [2]:
adv_dir = '/scratch/hle/fashion_mnist_adv/'
list_dirs = [f for f in os.listdir(adv_dir) if 'img' in f]
print((list_dirs[:10]))

['img_batch_0.pt', 'img_batch_1.pt', 'img_batch_2.pt', 'img_batch_3.pt', 'img_batch_4.pt', 'img_batch_5.pt', 'img_batch_6.pt', 'img_batch_7.pt', 'img_batch_8.pt', 'img_batch_9.pt']


In [3]:
test_imgs = []
for file_name in list_dirs:
    img_batch = torch.load(adv_dir + file_name).cpu().numpy()
    test_imgs.append(img_batch)
print(len(test_imgs))
test_imgs = np.concatenate(test_imgs)
test_imgs = np.transpose(test_imgs, [0,2,3,1])
print(test_imgs.shape)

157
(10000, 224, 224, 1)


In [4]:
model_path_test = '/scratch/hle/data/trained_models/fashion_mnist_gaussian_corrupted/fashion_mnist_gaussian_corrupted_model_46.ckpt'
img_size = (224,224,1)
BATCH_SIZE = 64
model = SARGAN(img_size, BATCH_SIZE, img_channel=1)
saver = tf.train.Saver()
gpu = "0"
gpu_options = tf.GPUOptions(allow_growth=True, visible_device_list=gpu)
config=tf.ConfigProto(gpu_options=gpu_options)
NOISE_STD = 0.1
recovered_imgs =[]
with tf.Session(config=config) as sess:
    saver.restore(sess, model_path_test)
    i = 0
    while i < 9984:
        batch = test_imgs [i:i + 64]
#         batch = [(std * image + mean) for image in batch]
        i += 64
        corrupted_batch = np.array([add_gaussian_noise(image, sd=NOISE_STD) for image in batch])
#         print("corrupted batch range %s, %s" % (np.amax(corrupted_batch), np.amin(corrupted_batch)))
        gen_imgs = sess.run(model.gen_img, feed_dict={model.image:batch, model.cond:corrupted_batch})
        recovered_imgs.append(gen_imgs)
    batch = test_imgs [9936:]
#     batch = [(std * image + mean) for image in batch]
    corrupted_batch = np.array([add_gaussian_noise(image, sd=NOISE_STD) for image in batch])
#         print("corrupted batch range %s, %s" % (np.amax(corrupted_batch), np.amin(corrupted_batch)))
    gen_imgs = sess.run(model.gen_img, feed_dict={model.image:batch, model.cond:corrupted_batch})
    gen_imgs = gen_imgs[-16:]
    recovered_imgs.append(gen_imgs)
    

W0905 02:05:04.349639 139825335146304 deprecation_wrapper.py:119] From /home/grad3/hle/sargan/sargan_models.py:13: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0905 02:05:04.354965 139825335146304 deprecation_wrapper.py:119] From /home/grad3/hle/sargan/sargan_models.py:46: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0905 02:05:04.356544 139825335146304 deprecation_wrapper.py:119] From /home/grad3/hle/sargan/utils.py:10: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0905 02:05:09.195943 139825335146304 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, p

In [9]:
i = 0
for batch in  recovered_imgs:
    torch.save(batch, "/scratch/hle/fashion_mnist_adv/img_recover_batch_%s.pt" % i)
    i+=1

In [11]:
print(len(list_dirs))

157
